### Monte-Carlo Tree Search with Neural Networks applied to play Checkers

#### Jair Taylor

Inspired in part by DeepMind's success with AlphaGo, we have written code that learns to play the game of Checkers using a somewhat similar methodology.  The algorithm uses a version of the Monte-Carlo Tree Search algorithm to learn find the proportion of wins that a good player should get from a given board state, and then trains a neural net to learn these probabilities.  

TODO:

1.  Create framework for evaluating strength of play. e.g., given an algorithm, what is its strength in terms of budget for a tree?
2.  Implement human-playable games.
3.  Create more general structure for DNN training.

In [1]:
from montecarlo_lib import *  # Here we have implemented the Monte-Carlo Tree Search algorithm 
                              # as well as defining the rules of Checkers.

import matplotlib.pyplot as plt

from ipywidgets import interact, IntSlider

from keras.models import Sequential
from keras.layers import Dense

%matplotlib inline

from pylab import rcParams

Using TensorFlow backend.


RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

RuntimeError: module compiled against API version 0xb but this version of numpy is 0xa

We begin by using MCTS to generate a large number of games of checkers.  No human-played games are used in training.

In [2]:
games_list = []
winners_list = []
all_game_trees_list = []
max_turns = 50
total_budget = 400
num_games = 20

for game_num in range(num_games):
    game = checkers_state(board_size = 6, max_turns = max_turns, tiebreaker_rule = True)
    #game.show_board()

    game_states_list = []
    game_trees_list = []

    print "Game %d commencing." % game_num
    for i in range(max_turns):
        num_actions = game.num_actions()     
        game.notes = ''
        game.notes += 'Turn %d: Player %d now choosing action from board above.\n' %  (i, game.player)
        game_to_play = deepcopy(game)
        game_to_play.max_turns = 60 #This setting stops stalling-for-time strategies
            
        game_tree = MonteCarloTree( deepcopy(game_to_play), 
                       budget = total_budget, 
                       num_simulations = 1, 
                       max_steps_to_simulate = 60)
        
        action = game_tree.UCTSearch()

        if not game_tree.root.is_complete and num_actions > 1 and len(game_tree.tree) < game_tree.budget:
            raise ValueError("Game tree not fully built")
        game_trees_list.append(deepcopy(game_tree))
        game.notes +=  'Size of tree: %d\n' % len(game_tree.tree)

        if action is None:
            game.notes +=  "No good move.  Taking random action.\n"
            action = game.random_action()
            
        game_states_list.append(deepcopy(game))
        (observation, reward, done, info) = game.step(action)        




        if done:
            winner = game.winner()
            
            if winner == 'draw':
                note = "Game is a draw."
            else:
                note = 'Player %d wins! (after %d moves)' % (game.winner(), i)
            game.notes = note
            print note
            winners_list.append(game.winner())
        #print game.notes
        #game.show_board()
        if done:
            game_states_list.append(deepcopy(game))
            break
    else:
        print "Game timed out."
        winners_list.append(None)
        
    games_list.append(deepcopy(game_states_list))
    all_game_trees_list.append(game_trees_list)
print winners_list



Game 0 commencing.
Game is a draw.
Game 1 commencing.
Player 1 wins! (after 37 moves)
Game 2 commencing.
Game is a draw.
Game 3 commencing.
Player 1 wins! (after 49 moves)
Game 4 commencing.


KeyboardInterrupt: 

Now we assemble the training set.  The training set consists of all nodes (that is, game states) in all the trees created above with depth below a fixed max_depth, together with the win/lose/draw probabilities for Player 0 as an output to be learned.



In [ ]:
all_X = []
all_Y = []

for game_index in range(num_games):
    game_states_list = games_list[game_index]
    game_trees_list = all_game_trees_list[game_index]
    
    for turn_index in range(len(game_trees_list)):
        game_tree = game_trees_list[turn_index]
        for node in game_tree.tree.values():
            
            results = node.all_simulation_results
            if len(results) > 0 and node.depth < 4:
                num_victories = results.count(1)
                num_losses = results.count(0)
                num_draws = results.count(0.5)

                if game_tree.root_player != 0:
                    num_victories, num_losses = num_losses, num_victories
                x = get_single_board_vector(node.state)
                totes = float(len(results))
                y = [num_victories/totes, num_losses/totes, num_draws/totes]
                #each y is: proportion of player 0 victories, player 1 victories, draws
                
                all_X.append(x)
                all_Y.append(y)
    
    
train_indices = get_random_subset(len(all_X), .8)

train_X = np.array([all_X[i] for i in range(len(all_X)) if i in train_indices])
train_Y = np.array([all_Y[i] for i in range(len(all_X)) if i in train_indices])

test_X = np.array([all_X[i] for i in range(len(all_X)) if i not in train_indices])
test_Y = np.array([all_Y[i] for i in range(len(all_X)) if i not in train_indices])

print 'train: %d / %d states (%.f pct)' % (len(train_X), len(all_X), 100 * len(train_X)/ float( len(all_X)))

print 'test: %d / %d states (%.f pct)' % (len(test_X), len(all_X), 100 * len(test_X)/ float( len(all_X))) 

We train a neural net model using Keras to learn these probabilities.

In [ ]:
layers = [30,20,10]
activations = ['relu', 'relu', 'relu']

model = Sequential()

for i in range(len(layers)):
    if activations is None:
        activation = 'relu'
    else:
        activation = activations[i]

    if i == 0:
        model.add(Dense(layers[i],  activation = activation, input_dim = train_X.shape[1]))
    else:
        model.add(Dense(layers[i],  activation = activation))
        
model.add(Dense(3, activation='softmax'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['mean_squared_error'])


# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae'])
# Fit the model


model.fit(train_X, train_Y, epochs=1000, batch_size=100, verbose = 2)
# evaluate the model
scores = model.evaluate(test_X, test_Y)

In [ ]:
scores = model.evaluate(test_X, test_Y, verbose = 0)

for i in range(len(model.metrics_names)):
    
    print "%s on test set: %f" % (model.metrics_names[i], scores[i])
    

Here we visualize the progress of a particular game of checkers and the associated win/lose/draw probabilites computed by our trained neural net.

In [ ]:
game_to_view = 0
game_states_list = games_list[game_to_view]
rcParams['figure.figsize'] = 4,4



def game_slider(i):
    game_states_list[i].show_board()
    
    v = get_single_board_vector(game_states_list[i])
    X = np.array([v])
    model_output = list(100 * model.predict(X)[0])
    print game_states_list[i].notes
    print "Player 0 victory: ", model_output[0], '%'
    print "Player 1 victory: ", model_output[1], '%'
    print "Draw:             ", model_output[2], '%'
    #return 'Player: %d' % game_states_list[i].player

interact(game_slider, i = IntSlider(min=0,max=len(game_states_list)-1,step=1,value=0)  )
None

Here we have a single step of a more general training algorithm to be implemented.  The idea is to simultaneously train the neural net while using the output from this model to improve the MCTS algorithm.

- At each step, we have previously determined a weight $\alpha \in [0,1]$ and a neural net f that takes as input a board state $B$ and outputs a triple $(w,l,d)$ corresponding to its estimate of winning, losing or drawing assuming "near-ideal" play.
- Play a large number of games using Monte-Carlo Tree Search.  
- During backpropogation, two 3-tuples are backpropogated:
    - $Q = (w, l, d)$ corresponding to the average number of wins, losses, and draws.
    - $\hat{Q}  = (\hat{w}, \hat{l}, \hat{d}) = f(B)$ corresponding to output of the previously-neural net $f$.
- For each node, set a value y corresponding to that node with $y = \alpha Q + (1-\alpha) \hat{Q}$ for some fixed $\alpha$. (Initially, set $\alpha = 1$; all weight is given to $Q$ when neural net is not yet trained.)
- The best child in MCTS is chosen according to the value of $y$.
- After many games have been played, create training set for neural net consisting of:
   - inputs $X$, game states corresponding to all nodes of depth < max_depth in all game trees
   - outputs $Y$, the the $y$ values given to these game states.
- Train neural net.
- Periodically, re-set the value of $\alpha$.  To find it, play a number of games between the following strategies: 
    - Strategy A: Choose moves by performing MCTS using random policy.
    - Strategy B: Choose moves by performing MCTS using scoring by $f$.

    Choose $\alpha$ to be the proportion of games won by Strategy A (plus half the proportion of games drawn.)
    
If the algorithm is succeeding, the value of $\alpha$ should gradually decrease on average.  Eventually, if $\alpha$ becomes small enough, set $\alpha = 0$ to save time (since evaluation of $\hat{Q}$ should be much faster than evaluation of $Q$.)

Periodically, the 'skill level' of neural net $f$ should be measured by the number of games won by Strategy A vs. Strategy B above, but with possibly with B given a handicap by having a large budget.  e.g., if $f$ is very accurate, Strategy A with a budget of 20 may be able to beat Strategy B with a budget of 100.



Other notes / questions - 

- Should at first play with small board size to see if training is working
- Over time, should increase the MCTS budget as $f$ becomes stronger
- Should Q-values of game trees be shared across turns or games? If so, how?